## Assignment 5 - Group 7

### First, we install all packages necesary and the chrome driver 

In [1]:
# Links 
# chromedriver= win 64: https://googlechromelabs.github.io/chrome-for-testing/
# chromedriver= win 32 :  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.109/win32/chromedriver-win32.zip

In [2]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver_manager==3.4.2

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Now, we import each library and install each package necesary to do the web scrapping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
# Now, we install the extention in order to access to the page we want
driver = webdriver.Chrome()

# We also maximize the window
driver.maximize_window()

# We create an empty list to store DataFrames
dataframes_list = []

# Here, we define the URL to be visited
url = 'https://infogob.jne.gob.pe/Eleccion'

# And open the specified URL in the Chrome browser
driver.get(url)

# Find and click on the dropdown element using XPath
dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
dropdown.click()

# Introduce a delay to wait for the dropdown options to load
time.sleep(1)

# Find and click on the option for presidential elections within the dropdown
elec_presidenciales = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
elec_presidenciales.click()

# Introduce a delay to wait for the next set of options to load
time.sleep(1)

# Find and click on the element corresponding to the election year
año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
año_eleccion.click()

# Introduce a delay to wait for the election year options to load
time.sleep(1)

# Find and click on the specific election year option using format
año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(2))
año_eleccion.click()

# Store the text content of the selected election year type
año_eleccion_tipo = año_eleccion.text

# Introduce a delay for stability before proceeding
time.sleep(1)


#####

# Get all elements of province
election_select = Select(driver.find_element(By.ID, "IdEleccion"))
num_election_options = len(election_select.options)

for elec_idx in range(1, num_election_options):
    # Get the election options again since HTML may be refreshing
    election_select = Select(driver.find_element(By.ID, "IdEleccion"))
    
    # Get the individual option
    election_option = election_select.options[elec_idx]

    # Get the option name of each election
    elec_name = election_option.get_attribute("text")
    
    print(elec_idx, elec_name)
    
    # Click the "Selecciona" button for each election
    selecciona = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    selecciona.click()
    time.sleep(2)

    # Navigate to the final page for each election
    pag_final = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    pag_final.click()
    time.sleep(2)
    
    table_path = driver.find_element(By.ID, 'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute ('innerHTML')
    table = pd.read_html(table_html)
    
    # Go back to the previous page
    driver.back()
    driver.back()

# Print all data as needed
 print(dataframes_list)


1 PRESIDENCIAL 2021 - 2DA VUELTA


ValueError: No tables found